<a href="https://colab.research.google.com/github/AlexZ-25/BlockchainDevelopment/blob/main/Curso%20Completo%20de%20Blockchain%20de%20cero%20a%20experto/Criptomoneda_5000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalaciones

In [2]:
!pip install flask==1.1.2

     |████████████████████████████████| 94 kB 1.7 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [3]:
!pip install requests==2.25.1

     |████████████████████████████████| 61 kB 2.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.25.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
!pip install flask-ngrok==0.0.25

# Mi Criptomoneda con Python

In [5]:
# Importación de las librerías
import datetime
import hashlib
import json
import requests
from uuid         import uuid4
from flask        import Flask, jsonify, request
from urllib.parse import urlparse
from flask_ngrok  import run_with_ngrok

Creación de una clase Blockchain que contenga todos los métodos esenciales, como son los siguientes:


*   Creación de un nuevo bloque
*   Obtención del hash de un bloque
*   Protocolo de concenso Proof of Work (PoW)
*   Generación del hash de un bloque
*   Verificación de la validez de la Blockchain
*   Añadir nueva transacción a la cadena de bloques
*   Añadir nuevo nodo a la cadena de bloques
*   Reemplazar la Blockchain por la adecuada



In [8]:
class Blockchain:

  def __init__(self):
    """ Constructor de la clase. """

    self.chain = []
    self.transactions = []
    self.create_block(proof = 1, previous_hash = '0')
    self.nodes = set()

  def create_block(self, proof, previous_hash):
    """ Creación de un nuevo bloque.

      Arguments:
        - proof: Nounce del bloque actual.
        - previous_hash: Hash del bloque previo.

      Returns:
        - block: Nuevo bloque creado.
    """

    block = { 'index'         : len(self.chain) + 1,
              'timestamp'     : str(datetime.datetime.now()),
              'proof'         : proof,
              'previous_hash' : previous_hash,
              'transactions'  : self.transactions}
    self.transactions = []
    self.chain.append(block)
    return block

  def get_previous_block(self):
    """ Obtención del bloque previo de la Blockchain.

        Returns:
          - Obtención del último bloque de la Blockchain. """

    return self.chain[-1]

  def proof_of_work(self, previous_proof):
    """ Protocolo de concenso Proof of Work (PoW).

      Arguments:
        - previous_proof: Nounce del bloque previo.

      Returns:
        - new_proof: Devolución del nuevo nounce obtenido con PoW. """
        
    new_proof = 1
    check_proof = False
    while check_proof is False:
      hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
      if hash_operation[:4] == "0000":
          check_proof = True
      else:
          new_proof += 1
    return new_proof

  def hash(self, block):
    """ Cálculo del hash de un bloque.

      Arguments:
        - block: Identifica a un bloque de la Blockchain.

      Returns:
        - hash_block: Devuelve el hash del bloque. """

    encoded_block = json.dumps(block, sort_keys= True).encode()
    hash_block = hashlib.sha256(encoded_block).hexdigest()
    return hash_block

  def is_chain_valid(self, chain):
    """ Determina si la Blockchain es válida.

      Arguments:
          - chain: Cadena de bloques que contiene la información 
                    de todas las transacciones.

      Returns:
          - True/False: Devuelve un booleano en función de la validez de la 
                        Blockchain. (True = Válida, False = Inválida)"""

    previous_block = chain[0]
    block_index = 1    
    while block_index < len(chain):
        block = chain[block_index]
        if block['previous_hash'] != self.hash(previous_block):
            return False
        previous_proof = previous_block['proof']
        proof = block['proof']
        hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] != '0000':
            return False
        previous_block = block
        block_index += 1
    return True

  def add_transaction(self, sender, receiver, amount):
    """ Realización de una transacción.

      Arguments:
        - sender: Persona que recibe la transacción.
        - receiver: Persona que recibe la transacción.
        - amount: Cantidad de criptomonedas enviadas.
      
      Returns:
        - Devolución del índice superior al último bloque
    """

    self.transactions.append({'sender'  : sender,
                              'receiver': receiver,
                              'amount'  : amount,})
    previous_block = self.get_previous_block()
    return previous_block['index'] + 1

    def add_note(self, address):
      """ Nuevo nodo en la Blockchain.

        Arguments:
          - address: Dirección del nuevo nodo.
      """

      parsed_url = urlparse(address)
      self.nodes.add(parsed_url.netloc)

    def replace_chain(self):
      """ Reemplazo de la cadena por la cadena más larga,
        siempre y cuando sea válida. """
      
      network = self.nodes
      longest_chain = None
      max_length = len(self.chain)
      for node in network:
        response = requests.get(f'http://{node}/get_chain')
        if response.status_code == 200:
          length = response.json()['length']
          chain = response.json()['chain']
          if length > max_length and self.is_chain_valid(chain):
            max_length = length
            longest_chain = chain
      if longest_chain:
        self.chain = longest_chain
        return True
      return False

In [ ]:
# Minado de un Bloque de la Cadena

# Crear una aplicación web
app = Flask(__name__)
run_with_ngrok(app)

# Si se obtiene un error 500, actualizar flask, reiniciar spyder y ejecutar la siguiente línea
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

# Crear